<a href="https://colab.research.google.com/github/aonurdasdemir/yahoo-nba-fantasy-ai-assistant/blob/main/NbaFantasyAIAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests oauthlib transformers

In [ ]:
import json
import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from datetime import datetime, timedelta

In [ ]:
def fetch_team_players(league_id, team_id, access_token):
    """
    Fetches the player IDs of the players in the given fantasy team.

    :param league_id: Yahoo Fantasy League ID
    :param team_id: Yahoo Fantasy Team ID
    :param access_token: OAuth2 access token
    :return: List of player IDs in the team
    """
    url = f"https://fantasysports.yahooapis.com/fantasy/v2/team/nba.l.{league_id}.t.{team_id}/roster/players?format=json"

    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None  # Return None if there's an error

    team_data = response.json()

    # Extract player IDs
    player_ids = []
    try:
        players = team_data["fantasy_content"]["team"][1]["roster"]["0"]["players"]
        for player_entry in players.values():
            if isinstance(player_entry, dict) and "player" in player_entry:
                player_info = player_entry["player"]
                player_id = player_info[0][0]["player_key"]
                player_ids.append(player_id)
    except KeyError:
        print("Error parsing team player data.")
        return None

    return player_ids

def fetch_top_player_keys(league_id, access_token, sort="AR", status="A", count=50):
    """
    Fetches the first `count` player keys sorted by the given criteria.

    :param league_id: Yahoo Fantasy League ID
    :param access_token: OAuth2 access token
    :param sort: Sorting criteria (default: "AR" for actual rank)
    :param status: Player status filter (default: "A" for all players)
    :param count: Number of players to fetch (default: 50)
    :return: List of player keys
    """
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }

    player_keys = []
    start = 0  # Yahoo returns 25 players per request, so we fetch with a for loop
    for i in range(count//25):
        url = f"https://fantasysports.yahooapis.com/fantasy/v2/league/nba.l.{league_id}/players;sort={sort};status={status};start={start}?format=json"

        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
        data = response.json()

        # Extract player list
        players = data["fantasy_content"]["league"][1]["players"]

        for count in range(25):
            player_key = players[str(count)]['player'][0][0]['player_key']
            player_keys.append(player_key)

        start += 25  # Move to the next batch

    return player_keys

def fetch_player_stats(league_id, access_token, player_keys, dates):
    """
    Fetch player statistics for multiple dates and return as a dictionary.

    :param league_id: Yahoo Fantasy League ID
    :param access_token: OAuth2 access token
    :param player_keys: List of player keys
    :param dates: List of dates (YYYY-MM-DD) for the statistics
    :return: Dictionary with player keys as top-level keys and stats for each date
    """
    stats_dict = {
        "9004003": "FGM/A",
        "5": "FG%",
        "9007006": "FTM/A",
        "8": "FT%",
        "10": "3PTM",
        "12": "PTS",
        "15": "REB",
        "16": "AST",
        "17": "ST",
        "18": "BLK",
        "19": "TO"
    }

    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }

    # Convert player_keys list to a comma-separated string
    player_keys_str = ",".join(player_keys)

    # Dictionary to store player stats
    player_stats_dict = {}

    for date in dates:
        url = f"https://fantasysports.yahooapis.com/fantasy/v2/league/nba.l.{league_id}/players;player_keys={player_keys_str}/stats;type=date;date={date}?format=json"

        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Error {response.status_code} for date {date}: {response.text}")
            continue  # Skip to the next date if an error occurs

        data = response.json()

        try:
            players = data["fantasy_content"]["league"][1]["players"]

            for player_entry in players.values():
                if isinstance(player_entry, dict) and "player" in player_entry:
                    player_info = player_entry["player"]

                    player_key = player_info[0][0]["player_key"]
                    player_full_name = player_info[0][2]["name"]["full"]
                    display_positions = player_info[0][11]["display_position"]
                    player_url = player_info[0][3]["url"]

                    # Initialize player entry if not present
                    if player_key not in player_stats_dict:
                        player_stats_dict[player_key] = {
                            "player_name": player_full_name,
                            "player_url": player_url,
                            "positions": display_positions,
                            "stats": {}
                        }

                    # Extract stats for this date
                    player_stats_dict[player_key]["stats"][date] = {}

                    for stat in player_info[1]["player_stats"]["stats"]:
                        stat_id = stat["stat"]["stat_id"]
                        stat_value = stat["stat"]["value"]
                        stat_name = stats_dict.get(stat_id, f"Stat {stat_id}")  # Map to human-readable name
                        player_stats_dict[player_key]["stats"][date][stat_name] = stat_value

        except KeyError:
            print(f"Error parsing player data for date {date}.")
            continue

    return player_stats_dict

def generate_last_n_days(n, end_date=None):
    """
    Generates a list of the last N days before the given end_date in 'YYYY-MM-DD' format.

    :param n: Number of past days to generate (including end_date)
    :param end_date: The last day in the interval (YYYY-MM-DD), default is today
    :return: List of dates as strings
    """
    if end_date:
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
    else:
        end_date = datetime.today()

    return [(end_date - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(n)]

To obtain `client_id` and `client_secret` for using the Yahoo Developer API in a Python notebook, follow these steps:

**1. Create a Yahoo Developer Account**

Go to the [Yahoo Developer Network.](https://developer.yahoo.com/)
Sign in using your Yahoo account. If you don’t have one, create a new Yahoo account.

**2. Create a New App**

Navigate to the [Yahoo Developer Apps](https://developer.yahoo.com/apps/).
Click "Create an App".
Fill in the required details:
Application Name: A name for your app.
Application URL: Can be any valid URL (e.g., http://localhost).
Description: Describe your app’s purpose.
API Permissions: Select the API you want to use (e.g., Fantasy Sports for Yahoo Fantasy Sports).
Redirect URI: Enter a valid redirect URL (e.g., https://localhost:8080).
OAuth Client Type: Choose "Confidential Client" option.

**3. Get client_id and client_secret**

After creating the app, you will be redirected to the app details page.
Under API Key Details, you will find:

Client ID: Your unique identifier.
Client Secret: A secret key used for authentication.

In [ ]:
# Your Yahoo API credentials
CLIENT_ID = input("Enter your client id: ")
CLIENT_SECRET = input("Enter your client secret: ")
TOKEN_URL = "https://api.login.yahoo.com/oauth2/get_token"

REDIRECT_URI = "oob"  # Out-of-band for manual copy-paste

AUTH_URL = "https://api.login.yahoo.com/oauth2/request_auth"
TOKEN_URL = "https://api.login.yahoo.com/oauth2/get_token"

# Yahoo OAuth session
oauth = OAuth2Session(CLIENT_ID, redirect_uri=REDIRECT_URI)

# Get authorization URL
auth_url, state = oauth.authorization_url(AUTH_URL)

print(f"Go to this URL and authorize the app:\n{auth_url}")



In [ ]:
AUTHORIZATION_CODE = input("Enter the authorization code from Yahoo: ")

# Exchange code for token
token_data = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "redirect_uri": REDIRECT_URI,
    "code": AUTHORIZATION_CODE,
    "grant_type": "authorization_code",
}

response = requests.post(TOKEN_URL, data=token_data)
token_info = response.json()

# Extract tokens
ACCESS_TOKEN = token_info.get("access_token")
REFRESH_TOKEN = token_info.get("refresh_token")

print(f"Access Token: {ACCESS_TOKEN}")
print(f"Refresh Token: {REFRESH_TOKEN}")


In [ ]:
def refresh_access_token(refresh_token):
    refresh_data = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "redirect_uri": REDIRECT_URI,
        "refresh_token": refresh_token,
        "grant_type": "refresh_token",
    }
    response = requests.post(TOKEN_URL, data=refresh_data)
    return response.json().get("access_token")

# Refresh token when needed
ACCESS_TOKEN = refresh_access_token(REFRESH_TOKEN)
print(f"New Access Token: {ACCESS_TOKEN}")


Enter your leage_id and team_id below. You can find these variables in the url when you click My Team. E.g. https://basketball.fantasysports.yahoo.com/nba/9515/5 Here 9515 corresponds to league_id and 5 corresponds to team_id.

In [ ]:
league_id = 9515 # Enter your own leage_id.
team_id = 5 # Enter the number of your team.
team_players = fetch_team_players(league_id, team_id, ACCESS_TOKEN)
top_players = fetch_top_player_keys(league_id, ACCESS_TOKEN, sort="AR", count=25)
team_players_stats = fetch_player_stats(league_id, ACCESS_TOKEN, team_players, generate_last_n_days(10))
top_available_players_stats = fetch_player_stats(league_id, ACCESS_TOKEN, top_players, generate_last_n_days(10))

In [ ]:
def generate_prompt(team_stats, top_players_stats):
    prompt = f"""
You are a Yahoo NBA Fantasy Basketball assistant. Analyze the recent performances of my team and suggest player moves.

### My Team's Players:
{json.dumps(team_stats, indent=2)}

### Top Available Free Agents:
{json.dumps(top_players_stats, indent=2)}

### Tasks:
1. Identify the weakest players on my team based on the last 10 days of stats.
2. Suggest which free agent players I should add, considering my team's weaknesses. Suggest me at least 5 players to add.
3. Consider balance across all stat categories (e.g., points, rebounds, assists, FG%, turnovers).
4. Provide a clear **Add/Drop** recommendation with reasoning.
"""
    return prompt

You can use your own Google AI Studio API key or choose any other LLM API.

In [ ]:
import google.generativeai as genai

genai.configure(api_key="") # Write your own key.
model = genai.GenerativeModel("gemini-1.5-flash")


In [ ]:
prompt = generate_prompt(team_players_stats, top_available_players_stats)
response = model.generate_content(prompt)

In [ ]:
print(response.text)

Based on the provided data, here's an analysis of your team and recommendations for player moves:

**1. Weakest Players:**

The data shows incomplete stats for the last 10 days, making a definitive assessment challenging. However, based on the available data, here's a ranking of players by recent performance (considering only games with available data):

* **Luke Kennard:** While he shows some decent assist numbers, his scoring is inconsistent and low.  His recent games have been a mix of decent output and very low.
* **De'Aaron Fox:**  Shows inconsistency in scoring and high turnover numbers.  His FG% is also worrisome in the games played.
* **Chris Paul:**  Similar to Fox; inconsistent scoring, but the assists are a strength. However, his scoring is lacking and the turnovers are an issue.
* **Julian Champagnie:** Shows inconsistent performance but has high upside in some games, particularly in 3-pointers and blocks.  The inconsistency is problematic though.
* **Tobias Harris:**  Gene